*Long-term benefits of nonpharmaceutical interventions for endemic infections are shaped by respiratory pathogen dynamics*

Rachel E. Baker, Chadi M. Saad-Roy, Sang Woo Park, Jeremy Farrar, C. Jessica E. Metcalf, and Bryan T. Grenfell


The authors start asking the effect of NPIs efforts applied during the 2020 and ongoing (2023 now) SARS2 pandemic in respiratory infections. They use influenza and RSV as contrasting pathogens with low and high basic reproductive number $R_0$ respectively and with rapid immune escape (SIRS dynamics) and completely immunizing (SIR dynamics).

In [1]:
import jax.numpy as np
from jax import grad, jit, vmap


def sirs(x, t, p):
    s, i, r = x

    β = p["beta"]
    γ = p["gamma"]
    ω = p["omega"]
    μ = p["mu"]
    N = p["N"]

    λ = β * i / N

    births  = μ * N
    s2death = μ * s
    i2death = μ * i
    r2death = μ * r

    s2i = λ * s
    i2r = γ * i
    r2s = ω * r

    sdot = -s2i + r2s - s2death + births
    idot = s2i  - i2r - i2death
    rdot = i2r  - r2s - r2death

    return np.array([sdot, idot, rdot])
